<a href="https://colab.research.google.com/github/Dinesh-Adhithya-H/Deep_Learning_using_Tensorflow/blob/main/Fashion_MNIST_using_Custom_Training_Loop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models  import Model
from tensorflow.keras.layers import Dense,Input
import pandas as pd
import tensorflow_datasets as tfds
import itertools
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from tqdm import tqdm

In [3]:
train_data, info = tfds.load("fashion_mnist", split = "train", with_info = True, data_dir='./data/', download=True)
test_data = tfds.load("fashion_mnist", split = "test", data_dir='./data/', download=False)

Shuffling and writing examples to ./data/fashion_mnist/3.0.1.incompleteE50IPW/fashion_mnist-train.tfrecord


Shuffling and writing examples to ./data/fashion_mnist/3.0.1.incompleteE50IPW/fashion_mnist-test.tfrecord


Dataset fashion_mnist downloaded and prepared to ./data/fashion_mnist/3.0.1. Subsequent calls will reuse this data.


In [4]:
def format_image(data):        
    image = data["image"]
    image = tf.reshape(image, [-1])
    image = tf.cast(image, 'float32')
    image = image / 255.0
    return image, data["label"]

In [5]:
train_data = train_data.map(format_image)
test_data = test_data.map(format_image)

In [6]:
batch_size = 64
train = train_data.shuffle(buffer_size=1024).batch(batch_size)
test =  test_data.batch(batch_size=batch_size)

In [7]:
def base_model():
  inputs=tf.keras.Input(shape=(784,),name='digits')
  x=tf.keras.layers.Dense(64,activation="relu",name="dense_1")(inputs)
  x=tf.keras.layers.Dense(64,activation="relu",name="dense_2")(x)
  outputs=tf.keras.layers.Dense(10,activation="softmax",name="prediction")(x)
  model=tf.keras.Model(inputs=inputs,outputs=outputs)
  return model

In [8]:
optimizer=tf.keras.optimizers.Adam()
loss_object=tf.keras.losses.SparseCategoricalCrossentropy()

In [9]:
train_acc_metric=tf.keras.metrics.SparseCategoricalAccuracy()
test_acc_metric=tf.keras.metrics.SparseCategoricalAccuracy()

In [10]:
def apply_gradient(optimizer,model,x,y):
  with tf.GradientTape as tape:
    logits=model(x)
    loss_value=loss_object(y_true=y,y_pred=logits)
  gradients=tape.gradient(loss_value,model.trainable_weights)
  optimizer.apply(zip(gradients,model.trainable_weights))
  return logits,loss_value

In [11]:
def train_data_for_one_epoch():
  losses = []
  pbar = tqdm(total=len(list(enumerate(train))), position=0, leave=True, bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} ')
  for step, (x_batch_train, y_batch_train) in enumerate(train):
      logits, loss_value = apply_gradient(optimizer, model, x_batch_train, y_batch_train)
      
      losses.append(loss_value)
      
      train_acc_metric(y_batch_train, logits)
      pbar.set_description("Training loss for step %s: %.4f" % (int(step), float(loss_value)))
      pbar.update()
  return losses

In [12]:
def perform_validation():
  losses = []
  for x_val, y_val in test:
      val_logits = model(x_val)
      val_loss = loss_object(y_true=y_val, y_pred=val_logits)
      losses.append(val_loss)
      val_acc_metric(y_val, val_logits)
  return losses

In [ ]:
model = base_model()

# Iterate over epochs.
epochs = 10
epochs_val_losses, epochs_train_losses = [], []
for epoch in range(epochs):
  print('Start of epoch %d' % (epoch,))
  
  losses_train = train_data_for_one_epoch()
  train_acc = train_acc_metric.result()

  losses_val = perform_validation()
  test_acc = val_acc_metric.result()

  losses_train_mean = np.mean(losses_train)
  losses_val_mean = np.mean(losses_val)
  epochs_val_losses.append(losses_val_mean)
  epochs_train_losses.append(losses_train_mean)

  print('\n Epoch %s: Train loss: %.4f  Validation Loss: %.4f, Train Accuracy: %.4f, Validation Accuracy %.4f' % (epoch, float(losses_train_mean), float(losses_val_mean), float(train_acc), float(val_acc)))
  
  train_acc_metric.reset_states()
  val_acc_metric.reset_states()
